<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

##  CPU Multi-Instrument Coupled Trigger Test

Test CPU Trigger to activate multi instrument coupled channel operation.
This script also allow testing of synchronizing tasks over multiple instruments without using change_dac_clock method (command ':TASK:SYNC')

Test Setup:
* 2x module of P948x
* Connect Sync cable between modules 1 and 2.
* Scope ch1 connected to channel 1 of module 1 and Scope ch2 connected to channel 1 of module 2.

This example creates 2 waves of different types, and downloads them into a task table to channel 1 on both instruments. The task's wait for an internal CPU trigger to activate the operation. The coupled cpu trigger mode activates all task tables together.
Run script and verify on Scope that:
- Before ('*TRG') command, both channels are not active
- After the cpu trigger command ('*TRG') signals are generated once on both channels.
- If signals are not synchronized after the *TRG command, you can run the ':TASK:SYNC' command in order to synchronize.


download the waves once for each DDR (depending on the number of channels in the device) on both instruments. Creates a task table. Download it for each channel and plays the task table according to a trigger send by the script. Verify on Scope connected to Channel 1 and 2 see generated signal after the '*TRG' command is run.

In [1]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [2]:
import os
import gc
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
#from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

In [3]:
# Connect to instrument

# Please choose appropriate address:
inst_addr = 'TCPIP::192.168.0.117::5025::SOCKET'

inst = TEVisaInst(inst_addr)

resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp)

connected to: Tabor Electronics,P9484M,000000000000,1.232.0 --Tabor Electronics,P9484M,000000000000,1.233.0 --slot#: 3, slot#: 5, 


In [4]:
inst.send_scpi_cmd(':INST:ACT 1')
inst.send_scpi_cmd(':XINS:SYNC:FOLL 1')

resp = inst.send_scpi_query(":SYST:ERR?")
print(resp)


0, no error


In [5]:
inst.send_scpi_cmd(':INST:ACT 1')
inst.send_scpi_cmd(':TRIG:COUPLE ON')
inst.send_scpi_cmd(':TRIG:CPU:MODE GLOBAL')
#inst.send_scpi_cmd(':INST:ACT 2')
#inst.send_scpi_cmd(':TRIG:COUPLE ON')

0

In [6]:
# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get model dependant parameters:

if model_name.startswith('P948'):
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    channels_per_dac = 2
elif model_name.startswith('P908'):
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    channels_per_dac = 1
else:
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    channels_per_dac = 2
    
half_dac = max_dac / 2.0

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = (int(resp) // 64) * 64
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))

max_seglen = arbmem_capacity // bpp
print ('Max segment length: {0:,}'.format(max_seglen))

Model: P9484M 
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 17,171,480,576 wave-bytes
Max segment length: 8,585,740,288


In [7]:
# Build 3 waveforms

seglen = 4096
cyclelen = seglen
ncycles = seglen / cyclelen
waves = [ None for _ in range(3)]

# sin wave:
x = np.linspace(
    start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
y = (np.sin(x) + 1.0) * half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
waves[0] = y.astype(wpt_type)

# triangle wave:
x = np.linspace(
    start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
y = np.sin(x)
y = np.arcsin(y)* 2 * half_dac / np.pi + half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
waves[1] = y.astype(wpt_type)

# square wave
x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
y = np.fmod(x, cyclelen)
y = (y <= cyclelen / 2) * max_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
waves[2] = y.astype(wpt_type)

In [8]:

# download 3 waveforms to each DDR
for insId in [1,2]:
    print('### Instrument #{} - segments'.format(insId))
    inst.send_scpi_cmd(':INST:ACT {}'.format(insId))
    channb = 1
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)
    for iseg in range(2):
        segnum = iseg + 1
        print('Downloading segment {0} of channel {1}'.format(segnum, channb))
        # Define segment
        cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
        inst.send_scpi_cmd(cmd)
        # Select the segment
        cmd = ':TRAC:SEL {0}'.format(segnum)
        inst.send_scpi_cmd(cmd)
        # Send the binary-data:
        inst.write_binary_data(':TRAC:DATA', waves[iseg])
            
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

### Instrument #1 - segments
### Instrument #2 - segments
0, no error


In [9]:
# Play the first segment in each channel
for insId in [1,2]:
    print('### Instrument #{} - play segment'.format(insId))
    inst.send_scpi_cmd(':INST:ACT {}'.format(insId))
    channb = 1
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)
    
    inst.send_scpi_cmd(':FUNC:MODE TASK')
    
    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

### Instrument #1 - play segment
### Instrument #2 - play segment
0, no error


In [10]:
# Define task-table of 3 tasks in each channel.
# The first task shall wait for trigger1.
# In order to 

tasklen = 2
for insId in [1,2]:
    print('### Instrument #{} - task table'.format(insId))
    inst.send_scpi_cmd(':INST:ACT {}'.format(insId))
# Compose the task-table rows:
    cmd = ':TASK:COMP:LENG {0}'.format(tasklen)
    inst.send_scpi_cmd(cmd)
    
    for itask in range(tasklen):
        tasknb = itask + 1
        segnb = itask + 1

        cmd = ':TASK:COMP:SEL {0}'.format(tasknb)
        inst.send_scpi_cmd(cmd)

        inst.send_scpi_cmd(':TASK:COMP:TYPE SING')

        cmd = ':TASK:COMP:SEGM {0}'.format(segnb)
        inst.send_scpi_cmd(cmd)

        if 1 == tasknb:
            # in case of :TRIG:COUPLE ON need to put INT instead of CPU??
            cmd = ':TASK:COMP:ENAB INT' 
            inst.send_scpi_cmd(cmd)
        else:
            cmd = ':TASK:COMP:ENAB NONE'
            inst.send_scpi_cmd(cmd)

        if tasklen == tasknb:
            cmd = ':TASK:COMP:NEXT1 1'
            inst.send_scpi_cmd(cmd)
        else:
            cmd = ':TASK:COMP:NEXT1 {0}'.format(tasknb + 1)
            inst.send_scpi_cmd(cmd)

    inst.send_scpi_cmd(':TASK:COMP:WRIT')

### Instrument #1 - task table
### Instrument #2 - task table


In [11]:
resp = inst.send_scpi_query(':FUNC:MODE?')
print(resp)

TASK


## Send CPU Trigger - 
Verify on Scope connected to Channel 1 and 2 and that signal is generated.

In [12]:
#inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':INST:ACT 1')
## Test Signal generatation from the task table.
inst.send_scpi_cmd('*TRG')


0

## option to sycronize tasks and channels over multiple instruments (without calling change_dac_clock)

In [13]:
if 1:
    inst.send_scpi_cmd(':TASK:SYNC')
    inst.send_scpi_cmd('*TRG')

## (Optional) Debug - Register Dump

In [14]:
import tempfile
import webbrowser

inst.send_scpi_cmd(':INST:ACT 2')
Debug=False

if Debug == True :
    channb = 1
    cmd = ':INST:CHAN {0}; :SYST:INF:REG?'.format(channb)
    html_str = inst.send_scpi_query(cmd, max_resp_len=200000)
    #print(html_str)
    with tempfile.NamedTemporaryFile('w', delete=False, suffix='.html') as f:
        url = 'file://' + f.name
        f.write(html_str)
    webbrowser.open(url)

In [15]:
inst.close_instrument()